In [1]:
import tensorflow as tf
import numpy as np
import random
import os
from multiprocessing import Process
import redis
import pickle
import subprocess
import keras

Using TensorFlow backend.


In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES']=""

In [3]:
redis_server_path = '/data/yylaiai/redis/redis-stable/src/redis-server'

In [4]:
# open redis server on current jupyter notebook terminal
subprocess.Popen([redis_server_path])

In [5]:
def load_model():
    model = keras.models.load_model('./alexnet-cifar10_origin.h5', compile=False) 
    return model

In [6]:
# load datasets
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
print(x_test.shape)

(10000, 32, 32, 3)


In [7]:
class GA:
    def __init__(self, fit, load_model, dataset, input_shape, init_input_mut, init_weight_mut, r1, r2, m, n, maxIter, db_flag):
        self.fitness_func, self.phi = fit # a tuple: (fitness function, list of DL framework(s) to be used)
        self.model = load_model()
        self.dataset = dataset
        self.input_shape = input_shape
        self.init_input_mut = init_input_mut
        self.init_weight_mut = init_weight_mut
        self.r1 = r1
        self.r2 = r2
        self.m = m
        self.n = n
        self.maxIter = maxIter
        self.db_flag = db_flag
        self.redis = redis.Redis(db=db_flag)
        
    def initPopulation(self):
        # randomly choose n inputs from the dataset if n < len(dataset)
        if len(self.dataset) > self.n:
            selected_x = self.dataset[np.random.choice(len(self.dataset), size=self.n, replace=False)]
        else:
            selected_x = self.dataset
            
        # get model weights
        original_weights = np.array(self.model.get_weights(), dtype=object)
            
        self.mutated_inputs = selected_x
        self.mutated_weights = original_weights

        # input level
        if self.init_input_mut != 0 and self.init_input_mut != None:
            self.mutated_inputs = (np.clip((selected_x/255 + np.random.standard_cauchy(selected_x.shape) * self.init_input_mut),0,1) * 255).astype(int)
            
        # weight level
        # set the model weights of the GA object to the mutated weights
        if self.init_weight_mut != 0 and self.init_weight_mut != None:
            self.mutated_weights = []
            for layer_weight in original_weights:
                self.mutated_weights.append(layer_weight + np.random.standard_cauchy(layer_weight.shape) * self.init_weight_mut)
            self.model.set_weights(self.mutated_weights)
                
        return [self.mutated_inputs, self.mutated_weights]


    def computeFitness(self):
        if self.fitness_func == "inc":
            FFunc = InconsistencyFunc("get_predicts.py", 0, self.redis, self.phi, self.model, self.mutated_inputs, -1)
            self.fitness_values = FFunc.compute()
            return self.fitness_values
        elif self.fitness_func == "nan":
            FFunc = NanFunc("get_predicts.py", 0, self.redis, self.phi, self.model, self.mutated_inputs, -1)
            self.fitness_values = FFunc.compute()
            return self.fitness_values
        
    def topK_fitness(self, k):
        return self.fitness_values[np.argpartition(self.fitness_values, -k)[-k:]]
        
        
    # select m candidates for parents of next-generation inputs
    def select(self):
        selected_index = np.argpartition(self.fitness_values, -self.m)[-self.m:]
        self.selected_x = self.mutated_inputs[selected_index]
        return self.selected_x
    
    # select 2 parents from the candidates
    def selectParents(self):
        self.x1, self.x2 = random.sample(list(self.selected_x), 2)
        return self.x1, self.x2
    
    # return a flatten list of a crossover product of the selected parents
    def crossover(self):
        x1_flatten = self.x1.flatten()
        x2_flatten = self.x2.flatten()

        x1_factor = np.random.choice(2, size=x1_flatten.shape, p=[0.5, 0.5])
        x2_factor = 1 - a
        
        self.x_prime = x1_flatten * x1_factor + x2_flatten * x2_factor

        return self.x_prime
    
    # mutate the crossover product and reshape it as the shape of the input instance
    def mutate(self):
        self.x_2prime=(np.clip((self.x_prime/255 + np.random.standard_cauchy(self.x_prime.shape) * self.r2),0,1) * 255).astype(int)
        self.x_2prime = self.x_2prime.reshape(self.input_shape)
        return self.x_2prime
    
    # check if the new DNN model can predict the mutated inputs without triggering error
    def checkFailed(self):
        try:
            if self.fitness_func == "inc":
                return []

            elif self.fitness_func == "nan":
                predictions = self.model.predict(self.mutated_inputs)
                if np.nan(predictions).any(): # if there is any nan in the predictions
                    return self.mutated_inputs
        except Exception as e:
            return [e]
            

In [8]:
class InconsistencyFunc:
    def __init__(self, py_script, db_flag, redis_server, backends, model, inputs, layer_idx):
        self.redis_server = redis_server
        self.db_flag = db_flag
        
        self.backend_1, self.backend_2 = backends
        self.model = model
        self.inputs = inputs
        self.cmd_1 = f'python {py_script} {self.backend_1} {db_flag} {layer_idx}'
        self.cmd_2 = f'python {py_script} {self.backend_2} {db_flag} {layer_idx}'
    
    def compute(self):
        # store model and inputs
        with self.redis_server.pipeline() as pipe:
            pipe.mset({"model": pickle.dumps(self.model)})
            pipe.mset({"inputs": pickle.dumps(self.inputs)})
            pipe.execute()
        
        # run subprocess to get predictions
        p1 = Process(target=lambda: os.system(self.cmd_1))
        p2 = Process(target=lambda: os.system(self.cmd_2))
        p1.start()
        p2.start()
        p1.join()
        p2.join()
        
        # load predictions
        with self.redis_server.pipeline() as pipe:
            pipe.hget(f"{self.backend_1}", "predictions")
            pipe.hget(f"{self.backend_2}", "predictions")
            predictions = pipe.execute()
        
        self.predictions_1 = pickle.loads(predictions[0])
        self.predictions_2 = pickle.loads(predictions[1])
        
        # compute fitness
        predictions_diff = np.abs(self.predictions_2 - self.predictions_1)
        
        self.fitness_values = np.sum(predictions_diff**2, axis=1) / len(self.predictions_1)
        
        return self.fitness_values


In [9]:
class NanFunc:
    def __init__(self, py_script, db_flag, redis_server, backend, model, inputs, layer_idx):
        self.redis_server = redis_server
        self.db_flag = db_flag
        
        self.model = model
        self.inputs = inputs
        self.backend = backend[0]
        self.cmd = f'python {py_script} {self.backend} {db_flag} {layer_idx}'
        
    def compute(self):
        # store model and inputs
        with self.redis_server.pipeline() as pipe:
            pipe.mset({"model": pickle.dumps(self.model)})
            pipe.mset({"inputs": pickle.dumps(self.inputs)})
            pipe.execute()
        
        # run subprocess to get predictions
        os.system(self.cmd)
        
        # load predictions
        self.predictions = pickle.loads(self.redis_server.hget(f"{self.backend}", "predictions"))
        
        # normalize neurons
        normalized_outputs = self.predictions/(np.amax(self.predictions) - np.amin(self.predictions))
        
        # compute fitness
        self.fitness_values = np.amax(normalized_outputs, axis=1) - np.amin(normalized_outputs, axis=1)
        
        return self.fitness_values


In [10]:
def ga_main(maxIter):
    F = []
    ga = GA(("inc", ["tensorflow", "theano"]), load_model, x_test[:100], x_test[0].shape, 0.1, 0, 0.5, 0.2, 5, 1000, 10, 0)
    x, theta = ga.initPopulation()
    for _ in range(maxIter):
        Fit = ga.computeFitness()
        P_prime = ga.select()
        test_cases = []
        test_cases.extend(P_prime)
        while len(test_cases) < ga.n:
            x1, x2 = ga.selectParents()
            x_prime = ga.crossover()
            r = random.uniform(0,1)
            if r < ga.r2:
                x_2prime = ga.mutate()
                test_cases.append(x_2prime)

        ga.mutated_inputs = np.array(test_cases)
        X = ga.checkFailed()
        if X != []:
            F.extend(X)
    return F

In [11]:
F = ga_main(1)

In [ ]:
ga = GA(("inc", ["tensorflow", "theano"]), load_model, x_test[:100], x_test[0].shape, 0.1, 0, 0.5, 0.2, 5, 1000, 10, 0)

In [ ]:
x, theta = ga.initPopulation()

In [ ]:
Fit = ga.computeFitness()

In [ ]:
P_prime = ga.select()

In [ ]:
test_cases = []
test_cases.extend(P_prime)
while len(test_cases) < ga.n:
    x1, x2 = ga.selectParents()
    x_prime = ga.crossover()
    r = random.uniform(0,1)
    if r < ga.r2:
        x_2prime = ga.mutate()
        test_cases.append(x_2prime)
        
ga.mutated_inputs = np.array(test_cases)

In [ ]:
X = ga.checkFailed()